In [18]:
import time

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
# from model import Model
# from femnist.cnn import ClientModel
# from utils.model_utils import read_data
import pydash
import math

pd.set_option('display.max_columns', 500)

In [3]:
IMAGE_SIZE = 28

In [21]:
num_clusters = 5
digits_per_cluster = np.array_split(range(10), num_clusters)
users_per_cluster = (len(users) // num_clusters) + 1

In [4]:
from collections import defaultdict
def read_dir(data_dir):
    clients = []
    groups = []
    data = defaultdict(lambda : None)

    files = os.listdir(data_dir)
    files = [f for f in files if f.endswith('.json')]
    for f in files:
        file_path = os.path.join(data_dir,f)
        with open(file_path, 'r') as inf:
            cdata = json.load(inf)
        clients.extend(cdata['users'])
#         for idx, u in enumerate(cdata['users']):
#             cluster_ids[u] = cdata['cluster_ids'][idx]
        if 'hierarchies' in cdata:
            groups.extend(cdata['hierarchies'])
            print("hierarchies exist")
        data.update(cdata['user_data'])

    clients = list(sorted(data.keys()))
#     cluster_ids = [cluster_ids[c] for c in clients]
    return clients, groups, data

In [5]:
users, _, data,  = read_dir('femnist-data/large/train')

In [12]:
_, _, data_test,  = read_dir('femnist-data/large/test')

In [13]:
complete_data = data.copy()
for username in users:
    complete_data[username]['x'].extend(data_test[username]['x'])
    complete_data[username]['y'].extend(data_test[username]['y'])

In [14]:
complete_data['f3478_49']['y'][0]

9

In [15]:
users[0]

'f0000_14'

In [23]:
def clean_and_split(data, test_split=0.1):
    cleaned = {}
    cleaned_test = {}
    cluster_ids = [None] * len(users)
    for user_index, username in enumerate(users):
        userdata = {}
        userdata['x']= []
        userdata['y']= []
        for index, y in enumerate(data[username]['y']):
            if y < 10 and y in [user_index%5 , user_index%5+5]:
                userdata['x'].append(data[username]['x'][index])
                userdata['y'].append(0 if y<5 else 1)
        if(len(userdata['y'])>0):
            train_size = math.floor(len(userdata['y']) * (1 - test_split))
            cleaned[username]= {'x': userdata['x'][:train_size],
                                'y': userdata['y'][:train_size]}
            cleaned_test[username]= {'x': userdata['x'][train_size:],
                                     'y': userdata['y'][train_size:]}
        else:
            print('Not enough data for client {}'.format(username))
        cluster_ids[user_index] = user_index%5
    return cleaned, cleaned_test, cluster_ids

In [24]:
cleaned, cleaned_test, cluster_ids = clean_and_split(complete_data)
cleaned['f3478_49']['y']

Not enough data for client f0785_26
Not enough data for client f1102_33
Not enough data for client f1107_44
Not enough data for client f1110_03
Not enough data for client f1723_34
Not enough data for client f1725_33
Not enough data for client f1726_01
Not enough data for client f1727_36
Not enough data for client f1969_24
Not enough data for client f1974_21
Not enough data for client f1984_28
Not enough data for client f1990_33
Not enough data for client f1992_08
Not enough data for client f1995_29
Not enough data for client f2044_20
Not enough data for client f2046_26


[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1]

In [27]:
train_output = {}
test_output = {} 
train_output['user_data'], test_output['user_data'], train_output['cluster_ids'] = clean_and_split(complete_data)
train_output['users'] = list(train_output['user_data'].keys())

test_output['cluster_ids'] = train_output['cluster_ids']
test_output['users'] = train_output['users']
with open('femnist-clustered/train/data.json', 'w') as file: 
    json.dump(train_output, file)
with open('femnist-clustered/test/data.json', 'w') as file: 
    json.dump(test_output, file)

Not enough data for client f0785_26
Not enough data for client f1102_33
Not enough data for client f1107_44
Not enough data for client f1110_03
Not enough data for client f1723_34
Not enough data for client f1725_33
Not enough data for client f1726_01
Not enough data for client f1727_36
Not enough data for client f1969_24
Not enough data for client f1974_21
Not enough data for client f1984_28
Not enough data for client f1990_33
Not enough data for client f1992_08
Not enough data for client f1995_29
Not enough data for client f2044_20
Not enough data for client f2046_26
